<a href="https://colab.research.google.com/github/deepak2233/Emoji-Prediction-Using-ML/blob/main/Emoji_Prediction_Using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install emoji

In [31]:
import numpy as np
import pandas as pd
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

In [32]:
train_data = pd.read_csv('/content/train_emoji.csv', header = None)
test_data = pd.read_csv('/content/test_emoji.csv',header =None)

In [33]:
train_data.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [34]:
import emoji as emoji

In [35]:

emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [36]:
emoji.emojize(":fork_and_knife:")


'🍴'

In [37]:
emoji.emojize(":baseball:")


'⚾'

In [38]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [39]:
data = train_data.values
for i in range(10):
    print(data[i][0],"-----", emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again ----- 😓
I am proud of your achievements ----- 😁
It is the worst day in my life ----- 😓
Miss you so much ----- ❤️
food is life ----- 🍴
I love you mum ----- ❤️
Stop saying bullshit ----- 😓
congratulations on your acceptance ----- 😁
The assignment is too long  ----- 😓
I want to go play ----- ⚾


In [40]:
from keras.utils import to_categorical

In [41]:
XT = train_data[0]
Xt = test_data[0]

YT = to_categorical(train_data[1])
Yt = to_categorical(test_data[1])


print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [42]:
embeddings = {}
with open('/content/glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [43]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [44]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [45]:
print(emb_XT.shape)
print(emb_Xt.shape)

(132, 10, 50)
(56, 10, 50)


In [46]:
from keras.layers import *
from keras.models import Sequential

In [47]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [48]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Epoch 1/40
4/4 [==============================] - 4s 283ms/step - loss: 1.6164 - acc: 0.2006 - val_loss: 1.6184 - val_acc: 0.0714
Epoch 2/40
4/4 [==============================] - 0s 23ms/step - loss: 1.5384 - acc: 0.4028 - val_loss: 1.6400 - val_acc: 0.0714
Epoch 3/40
4/4 [==============================] - 0s 23ms/step - loss: 1.5060 - acc: 0.3848 - val_loss: 1.6478 - val_acc: 0.3571
Epoch 4/40
4/4 [==============================] - 0s 24ms/step - loss: 1.4531 - acc: 0.4080 - val_loss: 1.6370 - val_acc: 0.3571
Epoch 5/40
4/4 [==============================] - 0s 23ms/step - loss: 1.4237 - acc: 0.4638 - val_loss: 1.5961 - val_acc: 0.2857
Epoch 6/40
4/4 [==============================] - 0s 22ms/step - loss: 1.3653 - acc: 0.4481 - val_loss: 1.5392 - val_acc: 0.1429
Epoch 7/40
4/4 [==============================] - 0s 21ms/step - loss: 1.3092 - acc: 0.4573 - val_loss: 1.4548 - val_acc: 0.2143
Epoch 8/40
4/4 [==============================] - 0s 22ms/step - loss: 1.2355 - acc: 0.5159 - va

In [49]:
model.evaluate(emb_Xt,Yt)

2/2 [==============================] - 0s 8ms/step - loss: 1.8058 - acc: 0.6429


[1.8058332204818726, 0.6428571343421936]

In [50]:
pred = model.predict_classes(emb_Xt)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [51]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
❤️
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😓
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😓
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
😁
I love taking breaks
❤️
❤️
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
😓
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁
